In [7]:
from __future__ import print_function
from __future__ import division

from collections import OrderedDict
import os
import sys
import warnings

import argparse
import logging
import h5py as h5
import numpy as np
import pandas as pd
import scipy.io

import six
from six.moves import range
import csv
import math as ma
from sklearn.metrics import roc_auc_score, confusion_matrix, average_precision_score,roc_curve,auc,precision_recall_curve
from keras.preprocessing import sequence
from keras.optimizers import RMSprop,Adam, Adadelta, Nadam, Adamax, SGD, Adagrad
from keras.models import Sequential
from keras.layers.core import  Dropout, Activation, Flatten
from keras.regularizers import l1,l2,l1_l2
from keras.constraints import maxnorm
#from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Conv1D, MaxPooling1D, Dense, LSTM, Bidirectional
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt

In [2]:
def features_selected_gbc(X_tr, y_tr):
	regr= GradientBoostingClassifier(random_state=0)
	regr.fit(X_tr, y_tr)
	coef = regr.feature_importances_
	parameters = {"coef": coef,
				  "model":regr}
	return parameters

In [3]:
h5filename = "histonemodTF_resample_ncl.h5"
h5file = h5.File(h5filename,'r')
input_features = h5file['input/H3K4me3_RPKM']
output_H3K4me3 = h5file['output/H3K4me3']
print(input_features.shape)
print(output_H3K4me3.shape)  

(26747, 43)
(26747,)


In [4]:
 input_features = np.array(input_features)
 output_H3K4me3 = np.array(output_H3K4me3)
 output_H3K4me3_reshape = output_H3K4me3.reshape(len(output_H3K4me3),1)

In [5]:
#combine the label with input dna
input_features_label = np.concatenate((input_features,output_H3K4me3_reshape), axis=1)
H3K4me3_df = pd.DataFrame(output_H3K4me3)
pos_label= H3K4me3_df.loc[H3K4me3_df.iloc[:,0]==1]
pos_label_ix = np.array(pos_label.index)
neg_label = H3K4me3_df.loc[H3K4me3_df.iloc[:,0]==0]
neg_label_ix = np.array(neg_label.index)
pos_sam_H3K4me3 = input_features_label[pos_label_ix,:]
neg_sam_H3K4me3 = input_features_label[neg_label_ix,:]
np.random.shuffle(pos_sam_H3K4me3)
np.random.shuffle(pos_sam_H3K4me3)
print(pos_sam_H3K4me3.shape)
print(neg_sam_H3K4me3.shape)

(9721, 44)
(17026, 44)


In [6]:
train_neg_sample = int(ma.ceil(neg_sam_H3K4me3.shape[0] *0.7))
train_pos_sample = int(ma.ceil(pos_sam_H3K4me3.shape[0] *0.7))
train_neg_H3K4me3 = neg_sam_H3K4me3[0:train_neg_sample,:]
train_pos_H3K4me3 = pos_sam_H3K4me3 [0:train_pos_sample,:]
train_neg_pos_H3K4me3 = np.concatenate((train_neg_H3K4me3, train_pos_H3K4me3),axis = 0)
np.random.shuffle(train_neg_pos_H3K4me3)
X_train_H3K4me3 = train_neg_pos_H3K4me3[:,0:30]
Y_train_H3K4me3 = train_neg_pos_H3K4me3[:,30]
Y_train_H3K4me3 = np.array(Y_train_H3K4me3, dtype='int64')
frq = np.bincount(Y_train_H3K4me3)
#print(frq)
print(X_train_H3K4me3.shape)
print(Y_train_H3K4me3.shape)
print(Y_train_H3K4me3[0:100])

(18724, 30)
(18724,)
[ 0  2  0  1  2  0  1  1  3  1  0  0  2  0  8  0  1  3  2  0  1  0  8  4  1
  0  5  4  0  3  0  0  0  6  1  1  2  1  5  1  3  1  1  0  0  8  2  0  0  0
  0  0  1  1  1  2  0  1  1  0  0  0  0 22  3  0  0  1  1  1  2  4  6  1 12
  6  1  0  0  0  0  0  0  0  7  1 16  0  0  1  0  1  0  0  0 11  1  1  2  1]


In [170]:
# feature selection using gbc
param = features_selected_gbc(X_train_H3K4me3, Y_train_H3K4me3)
coef = param["coef"]
coef = np.array(coef, dtype = 'float64')
print(np.count_nonzero(coef))
np.set_printoptions(formatter={'float_kind':'{:f}'.format})
print(coef)
b = np.sort(coef)
print(b)

30
[0.083487 0.035568 0.006085 0.029689 0.104137 0.011417 0.004659 0.062741
 0.003487 0.109399 0.039053 0.122498 0.041044 0.005618 0.006207 0.028472
 0.002622 0.010195 0.082123 0.002566 0.000006 0.036303 0.018983 0.049258
 0.030465 0.010066 0.014501 0.008607 0.024804 0.015939]
[0.000006 0.002566 0.002622 0.003487 0.004659 0.005618 0.006085 0.006207
 0.008607 0.010066 0.010195 0.011417 0.014501 0.015939 0.018983 0.024804
 0.028472 0.029689 0.030465 0.035568 0.036303 0.039053 0.041044 0.049258
 0.062741 0.082123 0.083487 0.104137 0.109399 0.122498]


In [171]:
features = coef>=0.028
X_train_H3K4me3 = X_train_H3K4me3[:,features]
print(X_train_H3K4me3.shape)

(18599, 14)


In [172]:
val_neg_sample = train_neg_sample + int(ma.ceil(neg_sam_H3K4me3.shape[0] *0.1))
val_pos_sample = train_pos_sample + int(ma.ceil(pos_sam_H3K4me3.shape[0] *0.1))
val_neg_H3K4me3 = neg_sam_H3K4me3[train_neg_sample:val_neg_sample,:]
val_pos_H3K4me3 = pos_sam_H3K4me3[train_pos_sample:val_pos_sample,:]
val_neg_pos_H3K4me3 = np.concatenate((val_neg_H3K4me3, val_pos_H3K4me3),axis = 0)
np.random.shuffle(val_neg_pos_H3K4me3)
X_val_H3K4me3 = val_neg_pos_H3K4me3[:,0:30]
Y_val_H3K4me3 = val_neg_pos_H3K4me3[:,30]
Y_val_H3K4me3 = np.array(Y_val_H3K4me3, dtype='int8')
frq = np.bincount(Y_val_H3K4me3)
print(frq)
print(X_val_H3K4me3.shape)
print(Y_val_H3K4me3.shape)   
print(X_val_H3K4me3.shape)
X_val_H3K4me3 = X_val_H3K4me3[:,features]
print(X_val_H3K4me3.shape)

[2410  247]
(2657, 30)
(2657,)
(2657, 30)
(2657, 14)


In [173]:
test_neg_H3K4me3 = neg_sam_H3K4me3[val_neg_sample:,:]
test_pos_H3K4me3 = pos_sam_H3K4me3 [val_pos_sample:,:]
test_neg_pos_H3K4me3 = np.concatenate((test_neg_H3K4me3, test_pos_H3K4me3),axis = 0)
np.random.shuffle(test_neg_pos_H3K4me3)
X_test_H3K4me3 = test_neg_pos_H3K4me3[:,0:30]
Y_test_H3K4me3 = test_neg_pos_H3K4me3[:,30]
Y_test_H3K4me3 = np.array(Y_test_H3K4me3, dtype='int8')
frq = np.bincount(Y_test_H3K4me3)
print(frq)
print(X_test_H3K4me3.shape)
print(Y_test_H3K4me3.shape) 
gbcmodel = param["model"]
y_pred = gbcmodel.predict(X_test_H3K4me3)
print('GBC model')
print(roc_auc_score(Y_test_H3K4me3, y_pred))
print(average_precision_score(Y_test_H3K4me3, y_pred))
X_test_H3K27me3 = X_test_H3K4me3[:, features]
print(frq)
print(X_test_H3K4me3.shape)
print(Y_test_H3K4me3.shape)
X_test_H3K4me3 = X_test_H3K4me3[:,features]
print(X_test_H3K4me3.shape)

[4819  494]
(5313, 30)
(5313,)
GBC model
0.762668729464
0.46716573349
[4819  494]
(5313, 30)
(5313,)
(5313, 14)


In [174]:
 model = Sequential()
 #model.add(Bidirectional(LSTM(30, return_sequences=True),input_shape=(30,1)))
 #model.add(Dropout(0.5))
 #model.add(Flatten())
 #model.summary()
 #model.add(Dense(units=256, input_dim=30, activation="relu", kernel_initializer='glorot_uniform'))
 #model.add(Dropout(0.1))
 model.add(Dense(units=256, input_dim=14, activation="relu",kernel_initializer='glorot_uniform')) 
 model.add(Dropout(0.3))
 model.add(Dense(units=180,  activation="relu",kernel_initializer='glorot_uniform'))
 model.add(Dropout(0.3))
 #model.add(Dropout(0.1))
 model.add(Dense(units= 60, activation="relu",kernel_initializer='glorot_uniform')) 
 #model.add(Dropout(0.8))
 #model.add(Dense(units=10, activation="relu",kernel_initializer='glorot_uniform'))
 model.add(Dense(units=1,  activation="sigmoid"))  
 #adam = SGD(lr=0.0001, momentum=0.95, nesterov=True)
 #adam = Nadam(lr = 0.0001)
 model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 256)               3840      
_________________________________________________________________
dropout_21 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_42 (Dense)             (None, 180)               46260     
_________________________________________________________________
dropout_22 (Dropout)         (None, 180)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 60)                10860     
_________________________________________________________________
dense_44 (Dense)             (None, 1)                 61        
Total params: 61,021
Trainable params: 61,021
Non-trainable params: 0
_________________________________________________________________


In [175]:
 adam = Adam(lr = 0.0001)
 model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
 print('running at most 60 epochs')
 checkpointer = ModelCheckpoint(filepath="HistoneMark_H3K4me3_TF_ncl_GM12878.hdf5",verbose=1, monitor='val_loss',save_best_only=True)
 earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
 model.fit(X_train_H3K4me3, Y_train_H3K4me3, batch_size=10, epochs=90, shuffle=True, validation_data=( X_val_H3K4me3, Y_val_H3K4me3), callbacks=[checkpointer,earlystopper])
 #model.fit(X_train_s, Y_train_s, batch_size=12, epochs=50, shuffle=True, validation_data=( X_val_s, Y_val_s), callbacks=[checkpointer,earlystopper])
 y_pred = model.predict(X_test_H3K4me3)
 
 np.savetxt('H3K4me3_true.csv', Y_test_H3K4me3, delimiter=",")
 np.savetxt('H3K4me3_pred.csv', y_pred, delimiter=",")
 #y_pred = model.predict(X_test_s)
 #tresults = model.evaluate(X_test_s, Y_test_s)
 tresults = model.evaluate(X_test_H3K4me3, Y_test_H3K4me3)
 print(tresults)
 model.summary()		
 #print(roc_auc_score(Y_test_s,y_pred))
 print(roc_auc_score(Y_test_H3K4me3, y_pred))
 print(average_precision_score(Y_test_H3K4me3, y_pred))


running at most 60 epochs
Train on 18599 samples, validate on 2657 samples
Epoch 1/90
18520/18599 [============================>.] - ETA: 0s - loss: 0.2669 - acc: 0.9157
Epoch 00001: val_loss improved from inf to 0.19657, saving model to HistoneMark_H3K4me3_TF_ncl_GM12878.hdf5
18599/18599 [==============================] - 7s 395us/step - loss: 0.2669 - acc: 0.9158 - val_loss: 0.1966 - val_acc: 0.9364
Epoch 2/90
18460/18599 [============================>.] - ETA: 0s - loss: 0.1963 - acc: 0.9316
Epoch 00002: val_loss improved from 0.19657 to 0.17471, saving model to HistoneMark_H3K4me3_TF_ncl_GM12878.hdf5
18599/18599 [==============================] - 7s 359us/step - loss: 0.1969 - acc: 0.9313 - val_loss: 0.1747 - val_acc: 0.9420
Epoch 3/90
18450/18599 [============================>.] - ETA: 0s - loss: 0.1819 - acc: 0.9353
Epoch 00003: val_loss improved from 0.17471 to 0.16682, saving model to HistoneMark_H3K4me3_TF_ncl_GM12878.hdf5
18599/18599 [==============================] - 7s 369u

18490/18599 [============================>.] - ETA: 0s - loss: 0.1187 - acc: 0.9550
Epoch 00056: val_loss improved from 0.13343 to 0.13294, saving model to HistoneMark_H3K4me3_TF_ncl_GM12878.hdf5
18599/18599 [==============================] - 6s 298us/step - loss: 0.1190 - acc: 0.9549 - val_loss: 0.1329 - val_acc: 0.9522
Epoch 57/90
18490/18599 [============================>.] - ETA: 0s - loss: 0.1212 - acc: 0.9557
Epoch 00057: val_loss improved from 0.13294 to 0.13225, saving model to HistoneMark_H3K4me3_TF_ncl_GM12878.hdf5
18599/18599 [==============================] - 5s 294us/step - loss: 0.1214 - acc: 0.9555 - val_loss: 0.1322 - val_acc: 0.9507
Epoch 58/90
18470/18599 [============================>.] - ETA: 0s - loss: 0.1206 - acc: 0.9556
Epoch 00058: val_loss did not improve
18599/18599 [==============================] - 6s 318us/step - loss: 0.1204 - acc: 0.9556 - val_loss: 0.1348 - val_acc: 0.9503
Epoch 59/90
18560/18599 [============================>.] - ETA: 0s - loss: 0.1182

In [ ]:
fpr, tpr, threshold = roc_curve(Y_test_H3K4me3, y_pred)
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
precision, recall, thresholds = precision_recall_curve(Y_test_H3K4me3, y_pred)
pr_auc = average_precision_score(Y_test_H3K4me3, y_pred)
plt.title('Precision Recall Curve')
plt.plot(recall, precision, 'b', label = 'AUC = %0.2f' % pr_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()
y_pred = (y_pred>0.5)
cm = confusion_matrix(Y_test_H3K4me3, y_pred)
print(cm)